# Cart Pole

#### Descripcion 


+ **Espacio de observacion (Box(4,))**

+ Posicion del carro: [-2.4, 2.4]
+ Velocidad del carro: [$-\infty$, $\infty$]
+ Angulo del palo: [-41.8, 41.8]
+ Velocidad del palo en la punta: [$-\infty$, $\infty$]


+ **Espacio de accion (Discrete(2))**

+ Izquierda: 0
+ Derecha: 1


El objetivo es mantener el palo vertical moviendo a izquierda y derecha el carro.




#### Perceptron implementado en Torch

In [1]:
import torch

class Perceptron(torch.nn.Module):
    
    def __init__(self, dim_entrada, dim_salida, hardware=torch.device('cpu')):
        super(Perceptron, self).__init__()                               # herencia
        self.hardware=hardware                                           # hardware usado por el perceptron
        self.dim_entrada=dim_entrada[0]                                  # dimension de la capa de entrada
        self.dim_oculta=50                                               # dimension de la capa oculta del perceptron (50 nodos, se puede cambiar)
        self.lineal=torch.nn.Linear(self.dim_entrada, self.dim_oculta)   # entrada (transformaacion lineal)
        self.salida=torch.nn.Linear(self.dim_oculta, dim_salida)         # salida
        
    def forward(self, x):  # metodo de creacion del perceptron
        x=torch.from_numpy(x).float().to(self.hardware)
        x=torch.nn.functional.relu(self.lineal(x)) # unidad rectificado lineal
        x=self.salida(x)
        return x

#### Clase Agente 

Se cambia la clase Agente para que los Q-valores sean obtenidos con el perceptron.

In [2]:
class Agente(object):
    
    def __init__(self, entorno):
        self.obs_dim=entorno.observation_space.shape              # dimension espacio observacion
        self.obs_sup=entorno.observation_space.high               # limite superior
        self.obs_inf=entorno.observation_space.low                # limite inferior
        self.obs_bins=NUM_BINS                                    # discretizacion de un espacio continuo, numero de bins
        self.ancho_bin=(self.obs_sup-self.obs_inf)/self.obs_bins  # ancho de cada parte de la discretizacion
        self.dim_accion=entorno.action_space.n                    # dimension espacio accion
        
        self.Q=Perceptron(self.obs_dim, self.dim_accion)                    # Q-valores
        self.Q_optimizador=torch.optim.Adam(self.Q.parameters(), lr=1e-5)   # optimizador Adam
        
        self.alfa=ALFA        # tasa de aprendizaje           
        self.gamma=GAMMA      # factor de descuento
        self.epsilon=EPSILON  # prob para escoger accion 
        
        
    def discretiza(self, obs):  # discretiza un espacio continuo
        return tuple(((obs-self.obs_inf)/self.ancho_bin).astype(int))  # binning
        
        
    def accion(self, obs):   # realiza la accion del agente
        obs_discreta=self.discretiza(obs) # discretiza espacio
        
        # politica epsilon-greedy para escoger la accion (mejor probabilidad 1-eps, peor es epsilon)
        if self.epsilon>MIN_EPSILON: self.epsilon-=DECAY_EPSILON
            
            
        if np.random.random()>self.epsilon: 
            return np.argmax(self.Q(obs_discreta).data.to(torch.device('cpu')).numpy())
        else: 
            return np.random.choice([a for a in range(self.dim_accion)])
        
        
    def aprende(self, obs, acciona, recompensa, siguiente_obs): # metodo de aprendizaje
        td_objetivo=recompensa+self.gamma*torch.max(self.Q(siguiente_obs))         # valor de aprendizaje
        td_error=torch.nn.functional.mse_loss(self.Q(obs)[acciona], td_objetivo)   # error aprendizaje
        self.Q_optimizador.zero_grad()
        td_error.backward()
        self.Q_optimizador.step()

In [3]:
import gym
entorno=gym.make('CartPole-v0')
for episodio in range(5):  # 5 episodios
    
    done=False
    
    observacion=entorno.reset()  # observacion
    
    recompensa_total=0.          # recompensa total en cada episodio
    
    paso=0                       # paso en cada episodio
     
    while not done:
        entorno.render()  # muestra el entorno
        
        accion=entorno.action_space.sample() # accion aleatoria, se cambiara por el agente
        
        siguiente_estado, recompensa, done, info=entorno.step(accion)
        
        recompensa_total+=recompensa
        
        paso+=1
        
        observacion=siguiente_estado
        
    print ('\nEpisodio #{} finalizado en {} pasos. Recompensa total:{}'.format(episodio+1, paso, recompensa_total))
    
entorno.close()


Episodio #1 finalizado en 14 pasos. Recompensa total:14.0

Episodio #2 finalizado en 15 pasos. Recompensa total:15.0

Episodio #3 finalizado en 17 pasos. Recompensa total:17.0

Episodio #4 finalizado en 20 pasos. Recompensa total:20.0

Episodio #5 finalizado en 21 pasos. Recompensa total:21.0
